In [1]:
import pandas as pd
import numpy as np
#import sample data
sample=pd.read_csv("sampledata_20082009")
sample.head()

,CREDIT_SCORE,FIRST_PAYMENT_DATE,FIRST_TIME_HOMEBUYER_FLAG,MATURITY_DATE,MSA,MI,NUMBER_OF_UNITS,OCCUPANCY_STATUS,CLTV,ORIGINAL_DTI,...,PRODUCT_TYPE,PROPERTY_STATE,PROPERTY_TYPE,POSTAL_CODE,LOAN_SEQUENCE_NUMBER,LOAN_PURPOSE,ORIGINAL_LOAN_TERM,NUMBER_OF_BORROWERS,SELLER_NAME,SERVICER_NAME
0,710,200803,N,203802,NaN,0,1,P,60,44,...,FRM,KY,SF,40400.0,F108Q1000136,C,360,1,Other sellers,Other servicers
1,752,200803,N,203802,38860.0,0,1,P,80,47,...,FRM,ME,SF,4000.0,F108Q1000216,C,360,1,Other sellers,Other servicers
2,738,200803,N,203802,NaN,35,1,P,100,13,...,FRM,IL,SF,61400.0,F108Q1000274,P,360,1,Other sellers,Other servicers
3,773,200803,Y,203802,45820.0,30,1,P,93,33,...,FRM,KS,SF,66600.0,F108Q1000309,P,360,1,Other sellers,Other servicers
4,813,200803,N,203802,NaN,0,1,P,74,31,...,FRM,IN,SF,46900.0,F108Q1000343,P,360,1,Other sellers,Other servicers


In [16]:
# Finding 1: After the financial meltdown, loans in general have high credit scores (FICO).(The financial crisis occurred in the third quarter of 2008)
before_crisis_FICO=round(sample[sample["FIRST_PAYMENT_DATE"]<200809].mean()["CREDIT_SCORE"])
after_crisis_FICO=round(sample[sample["FIRST_PAYMENT_DATE"]>200809].mean()["CREDIT_SCORE"])
print('Before the crisis, the average FICO was %3.0f. \nAfter the crisis, the average FICO was %3.0f.'% (before_crisis_FICO,after_crisis_FICO))

Before the crisis, the average FICO was 735. 
After the crisis, the average FICO was 757.


In [17]:
# Finding2: After the financial meltdown, the portion of subprime loans (FICO < 660) decreased substantially in the mortgage portfolio
NumSubprimeLoanAfter=len(sample[(sample["CREDIT_SCORE"]<660)&(sample["FIRST_PAYMENT_DATE"]>200809)])
NumAfterCrisis=len(sample[(sample["FIRST_PAYMENT_DATE"]>200809)])
AfterCrisisPortion=round(NumSubprimeLoanAfter/NumAfterCrisis,4)
AfterCrisisPortion
NumSubprimeLoanBefore=len(sample[(sample["CREDIT_SCORE"]<660)&(sample["FIRST_PAYMENT_DATE"]<=200809)])
NumBeforeCrisis=len(sample[(sample["FIRST_PAYMENT_DATE"]<=200809)])
BeforeCrisisPortion=round(NumSubprimeLoanBefore/NumBeforeCrisis,4)
BeforeCrisisPortion
print('Before the crisis, the portion of subprime loans was %2.2f%c. \nAfter the crisis, the portion of subprime loans was %2.2f%c.' % (BeforeCrisisPortion*100,'%',AfterCrisisPortion*100,'%'))

Before the crisis, the portion of subprime loans was 9.40%. 
After the crisis, the portion of subprime loans was 2.82%.


In [18]:
#Finding3: Before the financial meltdown, portion of purchases was higher.After the meltdown, portions of purchases went down. 
LoanPurchaseBefore=len(sample[(sample["LOAN_PURPOSE"]=='P')&(sample["FIRST_PAYMENT_DATE"]<=200809)])
NumBeforeCrisis=len(sample[(sample["FIRST_PAYMENT_DATE"]<=200809)])
Purchase_Portion_Before=LoanPurchaseBefore/NumBeforeCrisis
Purchase_Portion_Before
LoanPurchaseAfter=len(sample[(sample["LOAN_PURPOSE"]=='P')&(sample["FIRST_PAYMENT_DATE"]>200809)])
NumAfterCrisis=len(sample[(sample["FIRST_PAYMENT_DATE"]>200809)])
Purchase_Portion_After=LoanPurchaseAfter/NumAfterCrisis
Purchase_Portion_After
print('Before the crisis, the portion of purchases was %2.2f%c. \nAfter the crisis, the portion of purchases was %2.2f%c.' % (Purchase_Portion_Before*100,'%',Purchase_Portion_After*100,'%'))

Before the crisis, the portion of purchases was 33.28%. 
After the crisis, the portion of purchases was 26.74%.


In [20]:
#Finding4: Lower interest rates in 2009 also boosted the market for refinancing.
interestRate_2009=sample[(sample["FIRST_PAYMENT_DATE"]>=200901)&(sample["FIRST_PAYMENT_DATE"]<=200912)]["ORIGINAL_INTEREST_RATE"].mean()
interestRate_2009
interestRate_2008=sample[(sample["FIRST_PAYMENT_DATE"]>=200801)&(sample["FIRST_PAYMENT_DATE"]<=200812)]["ORIGINAL_INTEREST_RATE"].mean()
interestRate_2008
interestRate_2008>interestRate_2009
#refinance portion
Refinance_before_crises=len(sample[(sample["LOAN_PURPOSE"].isin (["C","N","R"]) )&(sample["FIRST_PAYMENT_DATE"]<=200809)])
Refinance_portion_before=Refinance_before_crises/NumBeforeCrisis
Refinance_after_crises=len(sample[(sample["LOAN_PURPOSE"].isin (["C","N","R"]) )&(sample["FIRST_PAYMENT_DATE"]>200809)])
Refinance_portion_after=Refinance_after_crises/NumAfterCrisis
print('Before the crisis, the portion of refinancing was %2.2f%c.The interest rate was %2.2f. \nAfter the crisis, the portion of refinancing was %2.2f%c.The interest rate was %2.2f' % (Refinance_portion_before*100,'%',interestRate_2008,Refinance_portion_after*100,'%',interestRate_2009))

Before the crisis, the portion of refinancing was 66.72%.The interest rate was 6.15. 
After the crisis, the portion of refinancing was 73.26%.The interest rate was 5.09


In [24]:
#Finding 5: The quality of loans originated after meltdown is considerably higher, i.e., their delinquency rate is much lower compared to those loans originated before the meltdown
#1. find loan sequence number in sample dataset
Loan_unique_id_before=sample[sample["FIRST_PAYMENT_DATE"]<200809]["LOAN_SEQUENCE_NUMBER"].unique()
len(Loan_unique_id_before)
Loan_unique_id_after=sample[sample["FIRST_PAYMENT_DATE"]>=200809]["LOAN_SEQUENCE_NUMBER"].unique()
len(Loan_unique_id_after)

13525

In [49]:
#2.find corresponding historical data of these loans in sample data
# load 2008Q1 performance historical data in chunk

In [25]:
# define a function to load bigdata in chunk
def read_big_data(path):
    mylist = []
    for chunk in  pd.read_csv(path, sep="|", chunksize=20000,
                         header=None,low_memory=False,
                         usecols=[0, 1,3, 7,18]):
        mylist.append(chunk)
    big_data = pd.concat(mylist, axis= 0)
    del mylist
    big_data.columns=["LOAN_SEQUENCE_NUMBER","MONTHLY_REPORTING_PERIOD","CURRENT_LOAN_DELINQUENCY_STATUS","MODIFICATION_FLAG","MAINTENANCE_AND_PRESERVATION_COSTS"]
    return big_data

In [36]:
# load 08Q1 performance data
big_data08Q1=read_big_data('historical_data1_time_Q12008.txt')
big_data08Q1_sample=big_data08Q1[(big_data08Q1["LOAN_SEQUENCE_NUMBER"].isin(Loan_unique_id_before))&(big_data08Q1["MONTHLY_REPORTING_PERIOD"]<=201303)]

In [37]:
#find delinquency loan in performance data sample
#delinquency criteria:
#1 loan has reached 90+ days in delinquency
#2 A loan has shown recovery amount as recovery usually comes after foreclosure and repossession by the bank (REO)
#3 A loan has been modified, i.e., interest rate reduction after negotiating with the bank.
delin_loan08Q1=big_data08Q1_sample[(big_data08Q1_sample['CURRENT_LOAN_DELINQUENCY_STATUS']==3)|(big_data08Q1_sample['MODIFICATION_FLAG']=='Y')|(big_data08Q1_sample['MAINTENANCE_AND_PRESERVATION_COSTS']>0)]['LOAN_SEQUENCE_NUMBER'].unique()
# number of delinquency loan in 08Q1 is 
delin_loan08Q1_num=len(delin_loan08Q1)
print('Number of delinquency loan in 08Q1 is', delin_loan08Q1_num)

Number of delinquency loan in 08Q1 is 112


In [26]:
# load 08Q2 performance data
big_data08Q2=read_big_data('historical_data1_time_Q22008.txt')
big_data08Q2_sample=big_data08Q2[(big_data08Q2["LOAN_SEQUENCE_NUMBER"].isin(Loan_unique_id_before))&(big_data08Q2["MONTHLY_REPORTING_PERIOD"]<=201306)]

In [27]:
delin_loan08Q2=big_data08Q2_sample[(big_data08Q2_sample['CURRENT_LOAN_DELINQUENCY_STATUS']==3)|(big_data08Q2_sample['MODIFICATION_FLAG']=='Y')|(big_data08Q2_sample['MAINTENANCE_AND_PRESERVATION_COSTS']>0)]['LOAN_SEQUENCE_NUMBER'].unique()
# number of delinquency loan in 08Q2 is 
delin_loan08Q2_num=len(delin_loan08Q2)
print('Number of delinquency loan in 08Q2 is', delin_loan08Q2_num)

Number of delinquency loan in 08Q2 is 64


In [29]:
# load 08Q3 performance data
big_data08Q3=read_big_data('historical_data1_time_Q32008.txt')

In [30]:
big_data08Q3_sample=big_data08Q3[(big_data08Q3["LOAN_SEQUENCE_NUMBER"].isin(Loan_unique_id_before))&(big_data08Q3["MONTHLY_REPORTING_PERIOD"]<=201309)]

In [32]:
#find delinquency loan in big data sample
delin_loan08Q3=big_data08Q3_sample[(big_data08Q3_sample['CURRENT_LOAN_DELINQUENCY_STATUS']==3)|(big_data08Q3_sample['MODIFICATION_FLAG']=='Y')|(big_data08Q3_sample['MAINTENANCE_AND_PRESERVATION_COSTS']>0)]['LOAN_SEQUENCE_NUMBER'].unique()
# number of delinquency loan in 08Q3 is 
delin_loan08Q3_num=len(delin_loan08Q3)
print('Number of delinquency loan in 08Q3 is', delin_loan08Q3_num)

Number of delinquency loan in 08Q3 is 2


In [40]:
#portion of delinquency loan before crisis
porDelinquencyBefore=(delin_loan08Q1_num+delin_loan08Q2_num+delin_loan08Q3_num)/len(Loan_unique_id_before)
porDelinquencyBefore
print("Delinquency rate before crisis is %1.4f"% porDelinquencyBefore)

Delinquency rate before crisis is 0.0395


In [ ]:
##### After crisis delinquency number 

In [44]:
# load 08Q4 performance data---after crisis
big_data08Q4=read_big_data('historical_data1_time_Q42008.txt')
big_data08Q4_sample=big_data08Q4[(big_data08Q4["LOAN_SEQUENCE_NUMBER"].isin(Loan_unique_id_after))&(big_data08Q4["MONTHLY_REPORTING_PERIOD"]<=201312)]

In [45]:
#find delinquency loan in 08Q4
delin_loan08Q4=big_data08Q4_sample[(big_data08Q4_sample['CURRENT_LOAN_DELINQUENCY_STATUS']==3)|(big_data08Q4_sample['MODIFICATION_FLAG']=='Y')|(big_data08Q4_sample['MAINTENANCE_AND_PRESERVATION_COSTS']>0)]['LOAN_SEQUENCE_NUMBER'].unique()
# number of delinquency loan in 08Q4 is 
delin_loan08Q4_num=len(delin_loan08Q4)
print('Number of delinquency loan in 08Q4 is ', delin_loan08Q4_num)

Number of delinquency loan in 08Q4 is  28


In [46]:
# load 09Q1 performance data---after crisis
big_data09Q1=read_big_data('historical_data1_time_Q12009.txt')

In [47]:
big_data09Q1_sample=big_data09Q1[(big_data09Q1["LOAN_SEQUENCE_NUMBER"].isin(Loan_unique_id_after))&(big_data09Q1["MONTHLY_REPORTING_PERIOD"]<=201403)]

In [48]:
#find delinquency loan in 09Q1
delin_loan09Q1=big_data09Q1_sample[(big_data09Q1_sample['CURRENT_LOAN_DELINQUENCY_STATUS']==3)|(big_data09Q1_sample['MODIFICATION_FLAG']=='Y')|(big_data09Q1_sample['MAINTENANCE_AND_PRESERVATION_COSTS']>0)]['LOAN_SEQUENCE_NUMBER'].unique()
# number of delinquency loan in 09Q1 is 
delin_loan09Q1_num=len(delin_loan09Q1)
print('Number of delinquency loan in 09Q1 is ', delin_loan09Q1_num)

Number of delinquency loan in 09Q1 is  18


In [49]:
# load 09Q2 performance data---after crisis
big_data09Q2=read_big_data('historical_data1_time_Q22009.txt')

In [50]:
big_data09Q2_sample=big_data09Q2[(big_data09Q2["LOAN_SEQUENCE_NUMBER"].isin(Loan_unique_id_after))&(big_data09Q2["MONTHLY_REPORTING_PERIOD"]<=201406)]

In [51]:
#find delinquency loan in 09Q2
delin_loan09Q2=big_data09Q2_sample[(big_data09Q2_sample['CURRENT_LOAN_DELINQUENCY_STATUS']==3)|(big_data09Q2_sample['MODIFICATION_FLAG']=='Y')|(big_data09Q2_sample['MAINTENANCE_AND_PRESERVATION_COSTS']>0)]['LOAN_SEQUENCE_NUMBER'].unique()
# number of delinquency loan in 09Q2 is 
delin_loan09Q2_num=len(delin_loan09Q2)
print('Number of delinquency loan in 09Q2 is ', delin_loan09Q2_num)

Number of delinquency loan in 09Q2 is  17


In [52]:
# load 09Q3 performance data---after crisis
big_data09Q3=read_big_data('historical_data1_time_Q32009.txt')

In [53]:
big_data09Q3_sample=big_data09Q3[(big_data09Q3["LOAN_SEQUENCE_NUMBER"].isin(Loan_unique_id_after))&(big_data09Q3["MONTHLY_REPORTING_PERIOD"]<=201409)]

In [54]:
#find delinquency loan in 09Q3
delin_loan09Q3=big_data09Q3_sample[(big_data09Q3_sample['CURRENT_LOAN_DELINQUENCY_STATUS']==3)|(big_data09Q3_sample['MODIFICATION_FLAG']=='Y')|(big_data09Q3_sample['MAINTENANCE_AND_PRESERVATION_COSTS']>0)]['LOAN_SEQUENCE_NUMBER'].unique()
# number of delinquency loan in 09Q3 is 
delin_loan09Q3_num=len(delin_loan09Q3)
print('Number of delinquency loan in 09Q3 is ', delin_loan09Q3_num)

Number of delinquency loan in 09Q3 is  9


In [55]:
# load 09Q4 performance data---after crisis
big_data09Q4=read_big_data('historical_data1_time_Q42009.txt')

In [56]:
big_data09Q4_sample=big_data09Q4[(big_data09Q4["LOAN_SEQUENCE_NUMBER"].isin(Loan_unique_id_after))&(big_data09Q4["MONTHLY_REPORTING_PERIOD"]<=201412)]

In [57]:
#find delinquency loan in 09Q4
delin_loan09Q4=big_data09Q4_sample[(big_data09Q4_sample['CURRENT_LOAN_DELINQUENCY_STATUS']==3)|(big_data09Q4_sample['MODIFICATION_FLAG']=='Y')|(big_data09Q4_sample['MAINTENANCE_AND_PRESERVATION_COSTS']>0)]['LOAN_SEQUENCE_NUMBER'].unique()
# number of delinquency loan in 09Q4 is 
delin_loan09Q4_num=len(delin_loan09Q4)
print('Number of delinquency loan in 09Q4 is ', delin_loan09Q4_num)

Number of delinquency loan in 09Q4 is  5


In [61]:
#portion of delinquency loan after crisis
porDelinquencyAfter=(delin_loan08Q4_num+delin_loan09Q1_num+delin_loan09Q2_num+delin_loan09Q3_num+delin_loan09Q4_num)/len(Loan_unique_id_after)
porDelinquencyAfter
print("Delinquency rate After crisis is %.4f"% porDelinquencyAfter)

Delinquency rate After crisis is 0.0057


In [63]:
print('Before the crisis, the delinquency rate was %.4f. \nAfter the crisis, the delinquency rate was %.4f.'% (porDelinquencyBefore,porDelinquencyAfter))

Before the crisis, the delinquency rate was 0.0395. 
After the crisis, the delinquency rate was 0.0057.
